# Neo4j Adapter Tutorial for Pydapter

This tutorial will show you how to use pydapter's Neo4j adapter to seamlessly
convert between Pydantic models and Neo4j graph databases. You'll learn how to
model, store, and query graph data using Pydantic's validation capabilities.

## A. Prerequisites

### A.1 Installation

the following command will

- create a virtual environment in the current directory and
- install the `pydapter` package with the `neo4j` extra dependencies.

In [ ]:
uv venv
uv pip install "pydapter[neo4j]"

### A.2 Set Up Neo4j

The easiest way to set up Neo4j is using Docker:

In [ ]:
docker run \
    --name neo4j-pydapter \
    -p 7474:7474 -p 7687:7687 \
    -e NEO4J_AUTH=neo4j/password \
    -d neo4j:latest

Alternatively, you can:

- Download and install Neo4j Desktop from
  [Neo4j's website](https://neo4j.com/download/)
- Use Neo4j AuraDB cloud service
- Install Neo4j directly on your system

With Docker, you can access:

- Neo4j Browser UI at http://localhost:7474
- Bolt protocol at bolt://localhost:7687


## B. Basic Example - Person Management System

### B.1 Store and Retrieve a Person Node from Neo4j

Import Required Libraries and Set Up Neo4j Config

In [1]:
from pydantic import BaseModel
from typing import List, Optional

# Neo4j connection settings
NEO4J_URI = "bolt://localhost:7687"
NEO4J_AUTH = ("neo4j", "password")  # Default credentials, change if different

NEO4J_CONFIG = {
    "url": NEO4J_URI,
    "auth": NEO4J_AUTH,
}

set up pydantic models

In [2]:
# Define a Pydantic model
class Person(BaseModel):
    id: str
    name: str
    age: int
    email: Optional[str] = None
    interests: List[str] = []

In [3]:
# Create some test data
people = [
    Person(
        id="p1",
        name="Alice",
        age=30,
        email="alice@example.com",
        interests=["coding", "hiking"],
    ),
    Person(
        id="p2",
        name="Bob",
        age=25,
        email="bob@example.com",
        interests=["gaming", "cooking"],
    ),
    Person(
        id="p3",
        name="Charlie",
        age=35,
        email="charlie@example.com",
        interests=["reading", "travel"],
    ),
]

In [4]:
NEO4J_PERSON_CONFIG = {**NEO4J_CONFIG, "label": "Person"}  # convience

In [5]:
from pydapter.extras.neo4j_ import Neo4jAdapter


# Store data in Neo4j
def store_people(people_list: List[Person]):
    print(f"Storing {len(people_list)} people in Neo4j...\n")

    for person in people_list:
        result = Neo4jAdapter.to_obj(
            person,
            merge_on="id",  # Property to use for MERGE operation
            **NEO4J_PERSON_CONFIG,
        )
        print(f"Stored {person.name}: {result}")

    print("\nPeople stored successfully.\n")


# Find people by property
def find_people_by_property(
    property_name: str = None, property_value: str = None, where_clause: str = None
):
    if not where_clause:
        if not property_name or not property_value:
            raise ValueError(
                "Either 'where_clause' or both 'property_name' and 'property_value' must be provided."
            )

    where_clause = where_clause or f"n.{property_name} = '{property_value}'"
    print(f"Finding people with {where_clause} ...")

    people: List[Person] = Neo4jAdapter.from_obj(
        Person,
        {"where": where_clause, **NEO4J_PERSON_CONFIG},
        many=True,
    )

    print(f"Found {len(people)} matching people:")
    for person in people:
        print(f"  - {person.name} (Age: {person.age}, Email: {person.email})")

    return people

In [6]:
def main():
    store_people(people)  # First, store people
    example_emails = find_people_by_property(
        where_clause="n.email ENDS WITH 'example.com'"
    )

    print(f"Found {len(example_emails)} people with example.com emails:")
    for person in example_emails:
        print(f"  - {person.name}: {person.email}")


main()

Storing 3 people in Neo4j...

Stored Alice: {'merged_count': 1}
Stored Bob: {'merged_count': 1}
Stored Charlie: {'merged_count': 1}

People stored successfully.

Finding people with n.email ENDS WITH 'example.com' ...
Found 3 matching people:
  - Alice (Age: 30, Email: alice@example.com)
  - Bob (Age: 25, Email: bob@example.com)
  - Charlie (Age: 35, Email: charlie@example.com)
Found 3 people with example.com emails:
  - Alice: alice@example.com
  - Bob: bob@example.com
  - Charlie: charlie@example.com


### B.2 Using pydapter Adaptable Mixin

add `Adaptable` mixin to the `Person` model

In [7]:
from pydapter import Adaptable
from pydapter.extras.neo4j_ import Neo4jAdapter


class Person(BaseModel, Adaptable):
    id: str
    name: str
    age: int
    email: Optional[str] = None
    interests: List[str] = []


Person.register_adapter(Neo4jAdapter)

In [8]:
people = [
  Person(
    id = "p1",
    name = "Alice",
    age = 30,
    email = "alice@example.com",
    interests = ["coding", "hiking"],
  ),
  Person(
    id = "p2",
    name = "Bob",
    age = 25,
    email = "bob@example.com",
    interests = ["gaming", "cooking"],
  ),
  Person(
    id = "p3",
    name = "Charlie",
    age = 35,
    email = "charlie@example.com",
    interests = ["reading", "travel"],
  ),
];

In [9]:
def main():
    # Store all people
    for person in people:
        person.adapt_to("neo4j", **NEO4J_PERSON_CONFIG)
    print("Stored all people in Neo4j.")

    # Find people with example.com emails
    example_emails = Person.adapt_from(
        {"where": "n.email ENDS WITH 'example.com'", **NEO4J_PERSON_CONFIG},
        obj_key="neo4j",
        many=True,
    )

    print(f"Found {len(example_emails)} people with example.com emails:")
    for person in example_emails:
        print(f"  - {person.name}: {person.email}")

### B.3 Working with Relationships

One of Neo4j's key features is its ability to model relationships between nodes.

In [12]:
from pydantic import BaseModel
from typing import List, Optional
from pydapter.extras.neo4j_ import Neo4jAdapter
from pydapter import Adaptable
from neo4j import GraphDatabase

# Neo4j connection settings
NEO4J_URI = "bolt://localhost:7687"
NEO4J_AUTH = ("neo4j", "password")


# Define models
class Person(BaseModel, Adaptable):
    id: str
    name: str
    age: int
    email: Optional[str] = None


class Hobby(BaseModel, Adaptable):
    id: str
    name: str
    category: Optional[str] = None


Person.register_adapter(Neo4jAdapter)
Hobby.register_adapter(Neo4jAdapter)


# Custom function to create relationships
# (Since pydapter doesn't directly handle relationships yet)
def create_relationship(person_id, hobby_id, relationship_type="ENJOYS"):
    """Create a relationship between a Person and a Hobby"""
    driver = GraphDatabase.driver(uri=NEO4J_URI, auth=NEO4J_AUTH)

    with driver.session() as session:
        result = session.run(
            f"""
            MATCH (p:Person {{id: $person_id}})
            MATCH (h:Hobby {{id: $hobby_id}})
            MERGE (p)-[r:{relationship_type}]->(h)
            RETURN p.name, h.name
            """,
            person_id=person_id,
            hobby_id=hobby_id,
        )

        for record in result:
            print(
                f"Created relationship: {record['p.name']} {relationship_type} {record['h.name']}"
            )

    driver.close()


# Function to find people who enjoy a specific hobby
def find_people_by_hobby(hobby_name):
    """Find all people who enjoy a specific hobby"""
    driver = GraphDatabase.driver(uri=NEO4J_URI, auth=NEO4J_AUTH)

    people_list = []

    with driver.session() as session:
        result = session.run(
            """
            MATCH (p:Person)-[:ENJOYS]->(h:Hobby {name: $hobby_name})
            RETURN p
            """,
            hobby_name=hobby_name,
        )

        for record in result:
            # Convert Neo4j node properties to dict
            person_data = dict(record["p"].items())
            # Create Pydantic model from data
            person = Person(**person_data)
            people_list.append(person)

    driver.close()
    return people_list


# Function to find hobbies for a specific person
def find_hobbies_for_person(person_id):
    """Find all hobbies for a specific person"""
    driver = GraphDatabase.driver(uri=NEO4J_URI, auth=NEO4J_AUTH)

    hobbies_list = []

    with driver.session() as session:
        result = session.run(
            """
            MATCH (p:Person {id: $person_id})-[:ENJOYS]->(h:Hobby)
            RETURN h
            """,
            person_id=person_id,
        )

        for record in result:
            hobby_data = dict(record["h"].items())
            hobby = Hobby(**hobby_data)
            hobbies_list.append(hobby)

    driver.close()
    return hobbies_list


# Main function to demo relationships
def main():
    # Create people
    people = [
        Person(id="p1", name="Alice", age=30, email="alice@example.com"),
        Person(id="p2", name="Bob", age=25, email="bob@example.com"),
        Person(id="p3", name="Charlie", age=35, email="charlie@example.com"),
    ]

    # Create hobbies
    hobbies = [
        Hobby(id="h1", name="Coding", category="Technical"),
        Hobby(id="h2", name="Hiking", category="Outdoor"),
        Hobby(id="h3", name="Reading", category="Indoor"),
        Hobby(id="h4", name="Cooking", category="Indoor"),
        Hobby(id="h5", name="Gaming", category="Entertainment"),
    ]

    # Store people in Neo4j
    print("Storing people...")
    for person in people:
        person.adapt_to(
            obj_key="neo4j",
            url=NEO4J_URI,
            auth=NEO4J_AUTH,
            label="Person",
            merge_on="id",
        )
    print("Stored all people in Neo4j.")

    # Store hobbies in Neo4j
    print("\nStoring hobbies...")
    for hobby in hobbies:
        hobby.adapt_to(
            obj_key="neo4j",
            url=NEO4J_URI,
            auth=NEO4J_AUTH,
            label="Hobby",
            merge_on="id",
        )

    # Create relationships
    print("\nCreating relationships...")

    # Alice enjoys Coding, Hiking, and Reading
    create_relationship("p1", "h1")
    create_relationship("p1", "h2")
    create_relationship("p1", "h3")

    # Bob enjoys Gaming and Cooking
    create_relationship("p2", "h4")
    create_relationship("p2", "h5")

    # Charlie enjoys Reading and Hiking
    create_relationship("p3", "h2")
    create_relationship("p3", "h3")

    # Find people who enjoy Hiking
    print("\nPeople who enjoy Hiking:")
    hikers = find_people_by_hobby("Hiking")
    for person in hikers:
        print(f"  - {person.name} (Age: {person.age})")

    # Find hobbies for Alice
    print("\nAlice's hobbies:")
    alice_hobbies = find_hobbies_for_person("p1")
    for hobby in alice_hobbies:
        print(f"  - {hobby.name} (Category: {hobby.category})")


main()

Storing people...
Stored all people in Neo4j.

Storing hobbies...

Creating relationships...
Created relationship: Alice ENJOYS Coding
Created relationship: Alice ENJOYS Hiking
Created relationship: Alice ENJOYS Reading
Created relationship: Bob ENJOYS Cooking
Created relationship: Bob ENJOYS Gaming
Created relationship: Charlie ENJOYS Hiking
Created relationship: Charlie ENJOYS Reading

People who enjoy Hiking:
  - Charlie (Age: 35)
  - Alice (Age: 30)

Alice's hobbies:
  - Reading (Category: Indoor)
  - Hiking (Category: Outdoor)
  - Coding (Category: Technical)


## Conclusion

In this tutorial, you've learned how to use pydapter's Neo4j adapter to
seamlessly work with graph databases. We've covered:

1. Basic setup and connection to Neo4j
2. Modeling entities as Pydantic models
3. Storing and retrieving data using the Neo4j adapter
4. Creating and traversing relationships
5. Building more complex graph applications
6. Error handling and best practices

Neo4j's graph structure is particularly powerful for data with complex
relationships, like social networks, recommendation systems, and knowledge
graphs. The pydapter adapter makes it easy to integrate Neo4j with your
Pydantic-based Python applications, providing a clean interface for graph
database operations.

Some key advantages of using pydapter's Neo4j adapter include:

1. Type safety and validation through Pydantic models
2. Consistent error handling
3. Simplified node creation and retrieval
4. Integration with other pydapter adapters for multi-database applications

Keep in mind that while the adapter handles nodes well, for relationship
operations you'll often need to use the Neo4j driver directly for more complex
graph traversals and Cypher queries.

To learn more about Neo4j and graph modeling, check out the
[Neo4j documentation](https://neo4j.com/docs/) and
[Cypher query language](https://neo4j.com/developer/cypher/).